In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
import json
from dotenv import load_dotenv
import re

In [2]:
# inference_path = 'ctr_valid_inference_mixtral.pkl'
inference_path = 'zeroshot_inference_test.json'
with open(inference_path, 'rb') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

6040


In [3]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [4]:
user_valid_movie_rating_df = ratings_df.groupby('user_id').nth(-1)
user_valid_movie_rating_df.head(10)

,user_id,movie_id,rating
52,1,48,5
453,10,2252,5
529,100,265,3
613,1000,1104,5
990,1001,647,3
1056,1002,1222,5
1085,1003,337,4
1566,1004,1266,5
1658,1005,266,1
1702,1006,2762,4


In [5]:
user_test_movie_rating_df = ratings_df.groupby('user_id').nth(-1)
user_test_movie_rating_df.head(10)

,user_id,movie_id,rating
52,1,48,5
453,10,2252,5
529,100,265,3
613,1000,1104,5
990,1001,647,3
1056,1002,1222,5
1085,1003,337,4
1566,1004,1266,5
1658,1005,266,1
1702,1006,2762,4


In [6]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 2309 -- No: 753 -- Invalid: 2978
invalid_users: ['10', '100', '1000', '1001', '1003', '1005', '1011', '1012', '1013', '1014', '1015', '1016', '1018', '1019', '102', '1021', '1023', '1025', '1026', '1029', '1030', '1032', '1033', '1035', '1037', '1038', '104', '1040', '1041', '1042', '1043', '1045', '1047', '1048', '105', '1050', '1051', '1055', '1056', '1059', '106', '1060', '1063', '1064', '1066', '1067', '1068', '1069', '107', '1072', '1074', '1077', '1079', '108', '1080', '1084', '1086', '1087', '1090', '1091', '1094', '1096', '1097', '1099', '11', '110', '1101', '1103', '1104', '1109', '1110', '1112', '1113', '1116', '1118', '1120', '1123', '1125', '1127', '1128', '1130', '1132', '1137', '1138', '1140', '1141', '1142', '1147', '1149', '115', '1150', '1152', '1153', '1158', '1159', '116', '1160', '1161', '1163', '1164', '1166', '1167', '1168', '1169', '1170', '1171', '1172', '1175', '1177', '1178', '1179', '118', '1181', '1184', '1187', '1194', '12', '120', '1200', '1201', '120

In [7]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    # original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user_id'] == user]['rating'].values[0]
    if len(user_test_movie_rating_df[user_test_movie_rating_df['user_id'] == user]) == 0:
        continue
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user_id'] == user]['rating'].values[0]
    # print(original_rating)
    if original_rating >= 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[]


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [13]:
movies_df = pd.read_csv('../movie_summary_25_words.csv')
movies_df[movies_df.movie_id == 1466]

,movie_id,name,cleaned_genre,year,summary
1439,1466,Donnie Brasco,"Crime,Drama",1997,"""Donnie Brasco"" is a gripping crime drama base..."


In [9]:
ctr_valid_dataset_path = './ctr_zeroshot_dataset/ctr_valid_zeroshot_dataset.pkl'
with open(ctr_valid_dataset_path, 'rb') as f:
    ctr_valid_data_dict = pickle.load(f)
print(len(ctr_valid_data_dict))

6040


In [14]:
print(ctr_valid_data_dict[1005])


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male lawyer between 35 to 44 years of age
            The user has a diverse taste in movies, with a preference for those that are thought-provoking and emotionally impactful. They enjoy movies that explore complex themes and blur the lines between reality and fantasy, as seen in their appreciation for "Being John Malkovich" and "Bowfinger." The user also has a soft spot for classic Christmas films, as evidenced by their fondness for "Miracle on 34th Street."
            
            In terms of genres, the user leans towards drama and comedy, with a particular interest in coming-of-age stories and character-driven narratives. They are drawn to movies that feature strong performances and explore the complexities of human relationships, as seen in their appreciation for "Diner" and "Schi

In [19]:
for user in invalid_users:
    print(user, inference_dict[user])

1164 
To analyze the user's behavior and predict whether they will like "Who Framed Roger Rabbit?" or not, we need to consider their movie watching history and preferences.

The user tends to enjoy movies that are character-driven, thought-provoking, and emotionally resonant. They have a clear preference for romantic comedies, comedies with wit and humor, and dramas that explore complex themes such as self-discovery, relationships, and the human condition.

Looking at the user's movie watching history, we can see that they have enjoyed movies such as "Mickey Blue Eyes," "The Mighty," "Out of Africa," "Rushmore," "Sleepy Hollow," "The Talented Mr. Ripley," "The Thomas Crown Affair," "Velvet Goldmine," and "Where the Heart Is." These movies all share common themes of love, relationships, and personal growth, with a strong focus on character development and storytelling.

On the other hand, the user has disliked movies such as "Midsummer Night's Dream," "Mystery Men," "Pushing Tin," "Stir